In [ ]:
import ipywidgets as widgets
from ipywidgets import interact, fixed

from mpl_toolkits.mplot3d import Axes3D

import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

## Multivariate linear regression

In [ ]:
# Generate multivariate data
np.random.seed(1)
n_samples = 100
sigma = np.sqrt(0.5)

x1 = np.random.rand(n_samples)
x2 = np.random.rand(n_samples)
X = np.matrix([np.ones(n_samples), x1, x2]).T # Design matrix

y = np.matrix(2*x1 + x2 + np.random.randn(n_samples)*sigma).T

In [ ]:
def plot_lr_3d(x1=None, x2=None, y=None, w0=0, w1=0, w2=0, show_mae=False, show_mse=False, show_errors=False):
    x1_min = np.min(x1)
    x1_max = np.max(x1)
    x2_min = np.min(x2)
    x2_max = np.max(x2)
    y_min = np.min(y)
    y_max = np.max(y)
    
    n = x1.shape[0]
    
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(x1, x2, y, c='b', linewidths=3, alpha=1)
    ax.set_xlabel('x1')
    ax.set_ylabel('x2')
    ax.set_zlabel('y')
    
    epsilon = 0.1
    ax.set_xlim(x1_min-epsilon, x1_max+epsilon)
    ax.set_ylim(x2_min-epsilon, x2_max+epsilon)
    ax.set_zlim(y_min-epsilon, y_max+epsilon)
    
    p = np.linspace(0, 1, 11)

    xx1, xx2 = np.meshgrid(p, p)

    z = w0 + w1*xx1 + w2*xx2

    ax.plot_surface(xx1, xx2, z)
    
    abs_err = 0
    sq_err = 0
    
    for i in range(len(y)):
        x1i = x1[i]
        x2i = x2[i]
        yi = y[i]
        pred_y = w0 + w1*x1i + w2*x2i
        if show_errors:
            ax.plot([x1i, x1i], [x2i, x2i], [yi, pred_y], c='r')
        abs_err += np.abs(yi - pred_y)
        sq_err +=  (yi - pred_y)**2
        
    if show_mae:
        print('Mean absolute error: %.3f' % (abs_err/n))
    if show_mse:
        print('Mean squared error: %.3f' % (sq_err/n))
    
    plt.show()

In [ ]:
%matplotlib notebook

ww0 = widgets.FloatSlider(min=-5, max=5)
ww1 = widgets.FloatSlider(min=-5, max=5)
ww2 = widgets.FloatSlider(min=-5, max=5)
show_errors = widgets.Checkbox(description='Show error bars')
show_mae = widgets.Checkbox(description='Show mean absolute error')
show_mse = widgets.Checkbox(description='Show mean squared error')

interact(plot_lr_3d, x1=fixed(x1), x2=fixed(x2), y=fixed(y), w0=ww0, w1=ww1, w2=ww2, show_errors=show_errors, show_mae=show_mae, show_mse=show_mse, ax=fixed(None));

Let's fit the multivariate regression model

In [ ]:
# Multiple the design matrix X with its transpose
XTX = X.T*X
XTX

In [ ]:
# Inverse XTX
XTX_inv = np.linalg.inv(XTX)
XTX_inv

In [ ]:
# Check that inverse gives reasonable results
XTX*XTX_inv

In [ ]:
# Compute the numerator
XTy = X.T*y
XTy

In [ ]:
# Compute the weights
w = XTX_inv*XTy
w

In [ ]:
# Plot points
%matplotlib notebook
#%matplotlib inline

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(x1, x2, y, c='b', linewidths=3, alpha=1)
ax.set_xlabel('x1')
ax.set_ylabel('x2')
ax.set_zlabel('y')
plt.show()

In [ ]:
# Plot the regression plane
p = np.linspace(0, 1, 11)

xx1, xx2 = np.meshgrid(p, p)

z = w[0, 0] + w[1, 0]*xx1 + w[2, 0]*xx2

ax.plot_surface(xx1, xx2, z)
plt.show()

In [ ]:
# Plot error bars
for i in range(n_samples):
    z = w[0, 0] + w[1, 0]*X[i,1] + w[2, 0]*X[i,2]
    ax.plot3D([X[i,1], X[i, 1]], [X[i,2], X[i,2]], [y[i], z], c='r')

In [ ]:
pred = (w.T*X.T).T
mse = mean_squared_error(pred, y)

print('MSE: %.3f' % mse)

In [ ]:
# Predict a point
new_point = np.matrix([1, 0.5, 0.5]).T
y_pred = (w.T*new_point)[0, 0]
y_pred

In [ ]:
# Plot new point
ax.scatter(new_point[1], new_point[2], y_pred, c='g', linewidths=6, alpha=1)